# 麻雀 Transformer AI 強化学習 (PPO + Custom Policy)

**目的:** 事前学習済みの Transformer モデルを基盤として、Stable Baselines3 の PPO アルゴリズムを用い、カスタム麻雀環境 (`MahjongEnv`) で自己対戦学習を実行します。

このノートブックは、Google Driveに配置されたPythonモジュールを呼び出し、学習を実行するためのランチャーです。

## ステップ1: 環境準備

### GPUの有効化
メニューから **「ランタイム」 > 「ランタイムのタイプを変更」** を選択し、**「ハードウェアアクセラレータ」** で **「GPU」** を選択してください。

In [ ]:
# 必要なライブラリのインストール
!pip install mahjong gymnasium "stable-baselines3[extra]" torch numpy

import torch

# GPUの利用状況を確認
if torch.cuda.is_available():
    device = "cuda"
    print(f"✅ GPUが利用可能です: {torch.cuda.get_device_name(0)}")
else:
    device = "cpu"
    print("❌ GPUが利用できません。CPUを使用します。")

## ステップ2: Google Driveのマウントとプロジェクトパス設定

自作のPythonモジュール (`MahjongEnv`, `MaskedTransformerPolicy` など) をインポートするために、Google Driveをマウントし、プロジェクトのルートディレクトリをPythonのパスに追加します。

In [ ]:
from google.colab import drive
import os
import sys

# Google Driveをマウント
drive.mount('/content/gdrive', force_remount=True)

# 【重要】ご自身の環境に合わせてプロジェクトのルートディレクトリを設定してください
PROJECT_ROOT = '/content/gdrive/MyDrive/MahjongEnhancedLeaning' # 例: '/content/gdrive/MyDrive/YourProjectFolder'

# 作業ディレクトリを設定し、パスに追加
if not os.path.exists(PROJECT_ROOT):
    raise FileNotFoundError(f"指定されたプロジェクトパスが見つかりません: {PROJECT_ROOT}")

os.chdir(PROJECT_ROOT)

if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"作業ディレクトリ: {os.getcwd()}")
print(f"Pythonパスに '{PROJECT_ROOT}' を追加しました。")

## ステップ3: モジュールのインポートと強化学習のセットアップ

Google Driveからカスタムモジュールをインポートし、強化学習のハイパーパラメータを設定してPPOモデルを初期化します。

In [ ]:
# --- 必要なモジュールをインポート ---
from stable_baselines3 import PPO
from mahjong_rl_env.environment import MahjongEnv
from models.custom_policy import MaskedTransformerPolicy

print("カスタムモジュールのインポートに成功しました。")

# --- ハイパーパラメータ設定 ---
RL_AGENT_ID = 0
FEATURES_DIM = 512
LEARNING_RATE = 3e-5
TOTAL_TIMESTEPS = 500_000  # 総学習ステップ数 (必要に応じて調整)
N_STEPS = 2048           # 1回の経験収集ステップ数

# --- 保存パスの設定 ---
TENSORBOARD_LOG_PATH = os.path.join(PROJECT_ROOT, "tensorboard_logs/")
MODEL_SAVE_PATH = os.path.join(PROJECT_ROOT, "models", "mahjong_ppo_agent.zip")

# 1. 環境のインスタンス化
env = MahjongEnv(agent_id=RL_AGENT_ID)
print(f"MahjongEnvを作成しました。RLエージェントはプレイヤー {RL_AGENT_ID}です。")

# 2. カスタムポリシーのための引数設定
policy_kwargs = dict(
    features_extractor_kwargs=dict(
        features_dim=FEATURES_DIM
        # ここでTransformerFeatureExtractorに渡す引数を追加可能
    ),
    # Actor/Critic の MLP 層の定義
    net_arch=[dict(pi=[256, 256], vf=[256, 256])]
)

# 3. PPOモデルの初期化
# 事前学習済み重みは、MaskedTransformerPolicy内のTransformerFeatureExtractorで自動的にロードされます。
model = PPO(
    MaskedTransformerPolicy,
    env,
    learning_rate=LEARNING_RATE,
    n_steps=N_STEPS,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    verbose=1,
    tensorboard_log=TENSORBOARD_LOG_PATH,
    device=device,
    policy_kwargs=policy_kwargs
)

print("PPOモデルとカスタムポリシーを初期化しました。")

## ステップ4: 強化学習の実行

設定した内容で強化学習を開始します。学習の進捗はTensorBoardで確認できます。

In [ ]:
print("--- 強化学習を開始します ---")
model.learn(total_timesteps=TOTAL_TIMESTEPS, progress_bar=True)
print("--- 学習が完了しました ---")

# 学習済みモデルの保存
model.save(MODEL_SAVE_PATH)
print(f"学習済みモデルを '{MODEL_SAVE_PATH}' に保存しました。")

## ステップ5: 学習済みモデルの評価

保存したモデルをロードし、エージェントがどのように行動するかをテストします。

In [ ]:
if os.path.exists(MODEL_SAVE_PATH):
    # モデルをロード
    loaded_model = PPO.load(MODEL_SAVE_PATH, env=env)
    print("モデルをロードしました。評価を開始します。")

    obs, info = env.reset()
    terminated = False
    total_reward = 0
    turn_count = 0

    while not terminated:
        turn_count += 1
        print(f"\n===== ターン {turn_count} =====")
        env.render()
        
        action, _states = loaded_model.predict(obs, deterministic=True)
        obs, reward, terminated, truncated, info = env.step(action)
        
        print(f"AIが行動 {action} を選択しました。 報酬: {reward}")
        total_reward += reward

        if terminated or truncated:
            print("\n--- 局が終了しました ---")
            print(f"最終スコア: {env.game_state['scores']}")
            print(f"この局の合計報酬: {total_reward}")
            break
else:
    print(f"保存されたモデルが見つかりません: {MODEL_SAVE_PATH}")